In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import numpy as np
from six.moves import range

In [ ]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars=sorted((set(chars)))
        self.char_indices=dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char=dict((i, c) for i, c in enumerate(self.chars))


    def encode(self, C, num_rows):
        x=np.zeros((num_rows, len(self.chars)))
        # One-hot encoding
        for i, c in enumerate(C):
            x[i, self.char_indices[c]]=1

        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x=x.argmax(axis=-1)

        return ''.join(self.indices_char[x] for x in x)


In [ ]:
class colors:
    ok='\033[92m'
    fail='\033[91m'
    close='\033[0m'


In [ ]:
TRAINING_SIZE=50000
DIGITS=3
REVERSE=True

MAXLEN=DIGITS+1+DIGITS

chars='0123456789+ '
ctable=CharacterTable(chars)

questions=[]
expected=[]
seen=set()


In [ ]:
print('Generating data')
while len(questions)< TRAINING_SIZE:
    # randomly generate a number within the limitation of length DIGITS
    f=lambda: int(''.join(np.random.choice(list('0123456789'))
        for i in range(np.random.randint(1, DIGITS+1))))

    a, b=f(), f()

    key=tuple(sorted((a, b)))
    if key in seen:
        continue

    seen.add(key)

    # construct the input sequence
    q='{}+{}'.format(a, b)
    # add space as padding
    query=q+' '*(MAXLEN-len(q))

    ans=str(a+b)
    ans+=' '*(DIGITS+1-len(ans))

    if REVERSE:
        query=query[::-1]

    questions.append(query)
    expected.append(ans)

print('Total addition questions: ', len(questions))


In [ ]:
print('Vectorization...')
x=np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y=np.zeros((len(questions), DIGITS+1, len(chars)), dtype=np.bool)

for i, sentence in enumerate(questions):
    x[i]=ctable.encode(sentence, MAXLEN)

for i, sentence in enumerate(expected):
    y[i]=ctable.encode(sentence, DIGITS+1)

# Shuffle the order of dataset
indices=np.arange(len(y))
np.random.shuffle(indices)
x=x[indices]
y=y[indices]

split_at=len(x)-len(x)//10
x_train, x_val=x[:split_at], x[split_at:]
y_train, y_val=y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

RNN=layers.LSTM
HIDDEN_SIZE=128
BATCH_SIZE=128
LAYERS=1


In [ ]:
print("ctable.chars", ctable.chars)
print("ctable.char_indices", ctable.char_indices)
print("ctable.indices_char", ctable.indices_char)
print("x[22]", x[22])
print("questions[22]", questions[indices[22]])
print("expected[22]", expected[indices[22]])
print("y[22]", y[22])

In [ ]:
print('Building model...')
model=Sequential()

model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS+1))


for _ in range(LAYERS):

    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:

for iteration in range(1, 200):
    print()
    print('-'*50)
    print('Iteration', iteration)
    model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=1, validation_data=(x_val, y_val))

    for i in range(10):
        ind=np.random.randint(0, len(x_val))
        rowx, rowy=x_val[np.array([ind])], y_val[np.array([ind])]
        preds=model.predict_classes(rowx, verbose=0)
        q=ctable.decode(rowx[0])
        correct=ctable.decode(rowy[0])
        guess=ctable.decode(preds[0], calc_argmax=False)

        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')

        if correct == guess:
            print(colors.ok+'☑'+colors.close, end=' ')
        else:
            print(colors.fail+'☒'+colors.close, end=' ')

        print(guess)